## 평가기준

1. CSR matrix가 정상적으로 만들어졌다.
   
  - 사용자와 아이템 갯수를 바탕으로 정확한 사이즈로 만들었다.
   
   
2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.

  - 사용자와 아이템 백터 내적 수치가 의미있게 형성되었다.
   
   
3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.


 - MF 모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.

#### 파일을 심볼릭 링크로 연결

In [1]:
! mkdir -p ~/aiffel/recommendata_iu/data/ml-1m
! ln -s ~/data/ml-1m/* ~/aiffel/recommendata_iu/data/ml-1m

# 데이터 준비와 전처리

### 데이터 준비

In [2]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


- 데이터 내용 부르기

- 4개의 항목으로 구성이 됨
: user_id , movie_id, rating, timestamp column

In [3]:

ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


- 평점이 3점이상인 movie는 전체 83.63%이다.

## 데이터 전처리

In [5]:
ratings.rename(columns={'rating':'count'}, inplace=True)

- count가 없기 때문에 count를 대신할 수 있는 rating의 이름을 변경한다.

In [6]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [7]:

movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


영화 제목을 확인하기 위해 데이터 부름.

In [8]:
ratings = ratings.join(movies.set_index('movie_id'), on='movie_id')
ratings.head()

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


별점 부분과 영화를 매칭함

허나 이번 프로젝트에는 장르와 timestamp가 필요없으므로 불필요한 데이터 삭제

In [9]:
ratings = ratings.drop(columns=['timestamp', 'genre'])
ratings.head()

,user_id,movie_id,count,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"


# 데이터 분석

1) ratings에 있는 유니크한 영화 개수

2) ratings에 있는 유니크한 사용자 수

3) 가장 인기 있는 영화 30개(인기순)

1. 유니크한 영화 개수, 영화제목 그리고 사용자 수 추출

In [10]:
print('# of movie_id: ', ratings['movie_id'].nunique())
print('# of title   : ', ratings['title'].nunique())
print('# of user_id : ', ratings['user_id'].nunique())

# of movie_id:  3628
# of title   :  3628
# of user_id :  6039


2. 가장 인기 있는 영화 30개순을 나열한다

-> 다양한 판단 기준이 있겠지만 저는 그 중에서 영화를 본 사용자의 수로 측정했습니다.

In [11]:
movie_count = ratings.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

사용자 기준으로 통계를 낸 이유는 가장 많은 유니크한 수를 가지고 있을 뿐만 아니라 가장 원초적으로 몇 만 관람수를 의미하기 때문입니다.

In [12]:
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

### 제가 유튜브에서 원하는 영화를 찾는 것처럼 선호 영화를 추가하겠습니다.

- 영화 제목을 키워드로 검색하기 입니다.

In [13]:

movies[movies['title'].str.lower().str.contains('star', regex=False)]

,movie_id,title,genre
122,124,"Star Maker, The (Uomo delle stelle, L') (1995)",Drama
129,131,Frankie Starlight (1995),Drama|Romance
195,197,"Stars Fell on Henrietta, The (1995)",Drama
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
313,316,Stargate (1994),Action|Adventure|Sci-Fi
325,329,Star Trek: Generations (1994),Action|Adventure|Sci-Fi
790,800,Lone Star (1996),Drama|Mystery
1025,1038,Unhook the Stars (1996),Drama
1178,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1192,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War


- 제가 직접 선호하는 영화 5편을 선정하여 dataframe을 만들기.

- 제가 선호하기 때문에 별점 최대인 5번을 본 것을 위주로 뽑겠습니다

In [14]:
my_favorite_id = [1,564, 1460 , 248, 3569]

my_favorite_title = []
for i in my_favorite_id:
    my_favorite_title.extend(list(movies[movies['movie_id'] == i]['title']))

my_movielist = pd.DataFrame({'user_id': ['Dutch_Lee']*5, 'movie_id': my_favorite_id, 'count': [5]*5, 'title': my_favorite_title})
my_movielist

,user_id,movie_id,count,title
0,Dutch_Lee,1,5,Toy Story (1995)
1,Dutch_Lee,564,5,Chasers (1994)
2,Dutch_Lee,1460,5,That Darn Cat! (1997)
3,Dutch_Lee,248,5,Houseguest (1994)
4,Dutch_Lee,3569,5,"Idiots, The (Idioterne) (1998)"


#### 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.


In [17]:
if not ratings.isin({'user_id':['Dutch_Lee']})['user_id'].any():
    ratings = ratings.append(my_movielist, ignore_index=True)

ratings.tail(15)

,user_id,movie_id,count,title
836468,6040,2021,3,Dune (1984)
836469,6040,2022,5,"Last Temptation of Christ, The (1988)"
836470,6040,2028,5,Saving Private Ryan (1998)
836471,6040,1080,4,Monty Python's Life of Brian (1979)
836472,6040,1089,4,Reservoir Dogs (1992)
836473,6040,1090,3,Platoon (1986)
836474,6040,1094,5,"Crying Game, The (1992)"
836475,6040,562,5,Welcome to the Dollhouse (1995)
836476,6040,1096,4,Sophie's Choice (1982)
836477,6040,1097,4,E.T. the Extra-Terrestrial (1982)


# CSR matrix

user_id, title를 고유 갯수만큼 인덱싱 과정을 거쳐야함.

In [18]:
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [19]:
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data
else:
    print('user_id column indexing Fail!!')

temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('title column indexing OK!!')
    ratings['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

ratings

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,count,title
0,0,1193,5,0
1,0,661,3,1
2,0,914,3,2
3,0,3408,4,3
4,0,2355,5,4
...,...,...,...,...
836478,6039,1,5,40
836479,6039,564,5,2433
836480,6039,1460,5,1988
836481,6039,248,5,546


### 눈에 보이는 변화

1. value of data column - > value of indexing


2. user_to_idx.get , movie_to_idx-> 인덱싱 -> user_id, title


## 최종적으로 CSR matrix생성

In [20]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['title'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

#### 이제 모델 설계  및 훈련을 해보도록하겠습니다.

In [22]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

implicit라이브러리가 권장을 하고 있는 부분입니다.

#### 모델 선언

In [23]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [26]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

- 행렬의 곱연산입니다.

-> 이 모델의 input는 item * user 꼴이기 때문에 행렬입니다. 그래서 Transpose를 이용해야합니다.

- 가볍게 모델 훈련하겠습니다.

In [25]:
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

#### 과연 훈련이된 모델이 나의 선호도를 어떻게 예측하고 있을까?

In [28]:
Dutch_Lee, matrix = user_to_idx['Dutch_Lee'], movie_to_idx['Toy Story (1995)']
Dutch_Lee_vector, matrix_vector = als_model.user_factors[Dutch_Lee], als_model.item_factors[matrix]

- model-> 나의 벡터와 영화벡터 구해야한다.

- 나의 벡터는 Dutch_Lee

- 선호 리스트에 있는 영화의 벡터 또한 계산을 해야 예측을 더 잘할 것 같습니다.


In [29]:
Dutch_Lee_vector

array([ 0.35591626,  0.44720066, -0.23267251,  0.00324943,  0.25925496,
        0.20701914,  0.40722203, -0.21954945,  0.2719917 ,  0.16185933,
       -0.05805085,  0.04739352, -0.00659016, -0.4108453 , -0.19799592,
       -0.04121689,  0.03730699,  0.05318306,  0.09071755, -0.24797344,
        0.363445  , -0.06958009,  0.16235138, -0.3310564 , -0.17535883,
       -0.26595834,  0.4991857 , -0.06826586, -0.04325735, -0.36860543,
       -0.04204901,  0.05586699,  0.05138635, -0.01978759,  0.09934324,
       -0.03087387, -0.12970951, -0.11040299,  0.2505709 ,  0.45581296,
       -0.4408402 , -0.26851854, -0.25649184, -0.05780477, -0.19693367,
        0.48630318,  0.5707374 , -0.17465012, -0.11844178,  0.10957062,
        0.31606206,  0.06888602, -0.37637722, -0.25124514, -0.04869989,
        0.16739932, -0.1560319 ,  0.6068209 ,  0.41115394,  0.7091253 ,
        0.0926566 ,  0.15251854, -0.08037585,  0.11589687, -0.5161434 ,
       -0.2918802 , -0.10723978,  0.16632503,  0.48052722,  0.29

In [30]:
matrix_vector

array([ 1.00669051e-02,  1.85698252e-02, -7.56696146e-03, -5.34281880e-03,
        3.02289929e-02,  2.09979508e-02,  2.17418261e-02, -1.49539588e-02,
        2.97710095e-02,  3.10908798e-02, -1.43186832e-02, -4.60338267e-03,
       -6.51683938e-03, -1.73139051e-02,  1.35334339e-02, -4.80277883e-03,
        1.38146402e-02,  1.00051537e-02, -2.90562143e-03, -2.56951079e-02,
        1.78354532e-02,  5.91290509e-03,  2.82626618e-02, -6.01043040e-03,
       -1.41535485e-02,  2.79107154e-03,  3.89596894e-02, -3.56738851e-03,
        2.34116614e-02, -2.34272648e-02,  2.01734826e-02,  2.13776976e-02,
       -4.54247747e-05,  1.24477025e-03,  1.31869381e-02,  1.06334984e-02,
        1.73593499e-02, -1.67543013e-02, -3.24992277e-03,  1.83025878e-02,
       -1.79383811e-02, -5.57743246e-04, -4.06714575e-03, -1.97164379e-02,
       -1.28886243e-03,  3.14489119e-02,  3.66430283e-02,  1.48076965e-02,
        9.74513032e-03, -8.62083025e-03,  3.19467448e-02, -1.44682061e-02,
        1.68050046e-03, -

내적을 하면 값이 나올텐데요 그 값은 아마 제 선호 리스트에 있기에 클거라고 생각합니다.

In [31]:
np.dot(Dutch_Lee_vector, matrix_vector)

0.44411603

그렇게 큰 숫자가 나온 것 같진 않지만 한 번에 제 리스트에 없는 영화와 비교해보아서 이 수치가 유의미한지에 대해서 판단해보겠습니다.

In [32]:
Superstar = movie_to_idx['Superstar (1999)']
Superstar_vector = als_model.item_factors[Superstar]
np.dot(Dutch_Lee_vector, Superstar_vector)

0.012403166

굉장히 낮은 값이 출력이 되었습니다

#### 미니 회고

제 선호 영화인 0.44 와 선호하지 않는 영화인 0.012의 내적값의 차이를 보니 제가 만든 모델이 예측을 못하지는 않는 것 같습니다.

### 이번에는 이 모델이 제가 좋아하는 영화와 비슷한 영화를 잘 추천해주는지 보겠습니다

- 마치 저희가 유튜브 알고리즘을 만드는 것과 같은거죠

좋아하는 영화와 비슷한 영화를 10개 정도만 출력해보겠습니다.

In [33]:
favorite_movie = 'Sixth Sense, The (1999)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=10)
similar_movie


[(38, 0.9999998),
 (233, 0.4969659),
 (121, 0.45671275),
 (243, 0.45243764),
 (170, 0.4171324),
 (832, 0.40359575),
 (273, 0.39266032),
 (220, 0.39196035),
 (81, 0.3916542),
 (1938, 0.3794605)]

In [34]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Sixth Sense, The (1999)',
 'Usual Suspects, The (1995)',
 'Silence of the Lambs, The (1991)',
 'Ghostbusters (1984)',
 'Being John Malkovich (1999)',
 'Jakob the Liar (1999)',
 'Fight Club (1999)',
 'Seven (Se7en) (1995)',
 'Green Mile, The (1999)',
 'Superstar (1999)']

idex를 이용하여 무슨 영화가 추천되었는지 볼 수 있도록 했습니다. 

#### 보고 싶은 영화가 매일 다를 수 있으니 함수화를 통해서 제목을 넣으면 상위 10개의 유사 영화가 나올 수 있도록 설정해보겠습니다.

In [35]:
def get_similar_movie(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id, N=10)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [36]:

get_similar_movie('Chasers (1994)')

['Chasers (1994)',
 'Bats (1999)',
 'Chill Factor (1999)',
 'Stay Tuned (1992)',
 'Trial and Error (1997)',
 'Firewalker (1986)',
 'Meatballs 4 (1992)',
 'Terminal Velocity (1994)',
 "National Lampoon's Senior Trip (1995)",
 'Almost Heroes (1998)']

In [37]:
get_similar_movie('That Darn Cat! (1997)')

['That Darn Cat! (1997)',
 'Meet the Deedles (1998)',
 'That Darn Cat! (1965)',
 'Dunston Checks In (1996)',
 'Gnome-Mobile, The (1967)',
 "We're Back! A Dinosaur's Story (1993)",
 "Blackbeard's Ghost (1968)",
 'Slappy and the Stinkers (1998)',
 'Harriet the Spy (1996)',
 'Flipper (1996)']

이렇게 하니 진짜 유튜브 알고리즘이 된 것 같아 신나네요

## 자 이제 제가 좋아할 만한!! 영화를 추천해주는 해보겠습니다.

먼저, 선호 영화와 비슷한 영화를 index값으로 반환해야합니다.

In [38]:

user = user_to_idx['Dutch_Lee']

movie_recommended = als_model.recommend(user, csr_data, N=15, filter_already_liked_items=True)
movie_recommended

[(50, 0.45874298),
 (4, 0.3025838),
 (322, 0.2834074),
 (33, 0.18303065),
 (110, 0.15148993),
 (474, 0.13694194),
 (10, 0.13673043),
 (160, 0.12566686),
 (330, 0.121911995),
 (255, 0.117742315),
 (284, 0.11289194),
 (476, 0.112036474),
 (475, 0.10707742),
 (1369, 0.0979512),
 (545, 0.09613222)]

인덱스 값을 바탕으로 선호 영화제목 출력

In [39]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Toy Story 2 (1999)',
 "Bug's Life, A (1998)",
 'Babe (1995)',
 'Aladdin (1992)',
 'Groundhog Day (1993)',
 'Chicken Run (2000)',
 'Beauty and the Beast (1991)',
 'Forrest Gump (1994)',
 'Lion King, The (1994)',
 "There's Something About Mary (1998)",
 'Nightmare Before Christmas, The (1993)',
 'Wrong Trousers, The (1993)',
 'My Cousin Vinny (1992)',
 '101 Dalmatians (1996)',
 'Santa Clause, The (1994)']

위의 매커니즘이 어떻게 제게 영화를 추천했을 때 도움을 주는지 확인해보겠습니다.

In [40]:
recommended = movie_to_idx["Bug's Life, A (1998)"]
explain = als_model.explain(user, csr_data, itemid=recommended)

[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Toy Story (1995)', 0.2849558996008382),
 ('Idiots, The (Idioterne) (1998)', 0.013490459854576193),
 ('Chasers (1994)', 0.005615083017542788),
 ('That Darn Cat! (1997)', 0.004001190649482695),
 ('Houseguest (1994)', -0.011281688055715751)]

영화 "Bug's Life, A (1998)" 를 통해서 'Toy Story (1995)'가 0.2849라는 수치가 나온 것으로 보아 많은 도움을 준 것 같습니다. 

그리고 신기한 것은 저는 Toy Story영화를 좋아합니다.  신기합니다!

# 프로젝트 회고!!

- 추천시스템은 이미 유튜브 알고리즘으로 인해서 익히 들었지만 실상 제대로 아는 것은 없었습니다. 허나 이번 익스를 통해서 어떻게 추천을 하고 우리가 그 추천에 빠져 시간을 사용하여 스트레스를 풀고 휴식을 하거나 아니면 시간 낭비가 되는 이유 또한 알게 되어서 재미난 경험이었습니다. 이 시스템은 점점 OTT플랫폼을 이용하는 회사들이라면 더욱 발전시키고 공을 들여야만 이용자들이 원하고 회사에게도 경제적 이득을 줄 것 같습니다.!




- 익스의 경험치가 쌓이다보니 이제 데이터를 먼저 보고 무엇이 필요하고 무엇이 불필요한지 구분이 되는 것 같아서 점점 기분이 좋습니다. 무튼 이 경험을 통해서 평점과 제목이 나눠진 데이터를 dataframe을 통해서 하나로 합쳤습니다. 그러면 유사성을 보기 편할 것 같아서요!




- 익스의 결과를 보면, 선호 영화는 내적 값이 0.44로 높은 값을 가지지만 선호하지 영화에서는 값이 0.012로 나타나는 것으로 보아 모델이 학습이 잘 되었고 선호도 반영을 잘한 것 같습니다.




- 그리고 신기하게 좋아하는 영화를 추천해줄 때 실제로 제가 선호하는 영화의 유사한 장르를 바탕으로 추천을 해준 것 같아서 너무 신기했고 신뢰도가 쌓였습니다.